In [18]:
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import tensorflow as tf
from scipy.signal import convolve2d
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

data = load_data()

train_data = data[0][0]
train_answer = data[0][1]

#kernels
kernel_horizontal = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernel_vertical = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
kernel_oblique1 = np.array([[1,0,-1],[0,1,0],[-1,0,1]])
kernel_oblique2 = np.array([[-1,0,1],[0,1,0],[1,0,-1]])

#onehotencoding for training the number 5
answer = [1 if i==5 else 0 for i in train_answer]

# This function is taylored for evenly shaped images
def maxpool(image):
    #we check if the image's size is a multiple of three
    if ((image.shape[0]%3 + image.shape[1]%3) == 0):
        new_image = np.zeros((int(image.shape[0]/3),int(image.shape[1]/3)))
        for i in range(int(image.shape[0]/3)):
            for j in range(int(image.shape[1]/3)):
                new_image[i,j]= max(image[3*i, 3*j:3*j+3].max(),image[3*i+1,3*j:3*j+3].max(), image[3*i+2,3*j:3*j+3].max())
        return new_image
    return image.shape[0]%3 + image.shape[1]%3

#activation function
def sigmoid(k,x):
    return 1/(1+tf.math.exp(-tf.linalg.matmul(k,x)))


coefs = np.random.randn(5, 1, 9*9)

coefs = tf.Variable(coefs)

loss_counter = []

for i in range(10):
    for j in range(100):
        with tf.GradientTape(persistent= True) as tape:
            tape.watch(coefs)

            #convolving in all directions
            convolved_vert = convolve2d(train_data[j], kernel_vertical, mode= 'same')
            convolved_hor = convolve2d(train_data[j], kernel_horizontal, mode= 'same')
            convolved_ob1 = convolve2d(train_data[j], kernel_oblique1, mode= 'same')
            convolved_ob2 = convolve2d(train_data[j], kernel_oblique2, mode= 'same')

            #maxpool applied on 27x27 convolved image
            pooled_vert = maxpool(convolved_vert[1:,1:])
            pooled_hor = maxpool(convolved_hor[1:, 1:])
            pooled_ob1 = maxpool(convolved_ob1[1:,1:])
            pooled_ob2 = maxpool(convolved_ob2[1:,1:])

            #flattening
            flat_vert = pooled_vert.reshape(1,9*9,1)
            flat_hor = pooled_hor.reshape(1,9*9,1)
            flat_ob1 = pooled_ob1.reshape(1,9*9,1)
            flat_ob2 = pooled_ob2.reshape(1,9*9,1)

            #outputs in the hidden layer
            hidden1 = sigmoid(coefs[0].reshape(1,1,9*9), flat_vert)
            hidden2 = sigmoid(coefs[1].reshape(1, 1, 9 * 9), flat_hor)
            hidden3 = sigmoid(coefs[2].reshape(1, 1, 9 * 9), flat_ob1)
            hidden4 = sigmoid(coefs[3].reshape(1, 1, 9 * 9), flat_ob2)

            hidden_vector = tf.constant([int(hidden1),int(hidden2), int(hidden3), int(hidden4)], shape= (4,1))

            #output
            out = 1/(1+tf.math.exp(-coefs[4][0][:4]@hidden_vector))

            #loss
            loss = -answer[j]*tf.math.log(out)-(1-answer[j])*tf.math.log(1-out)

        #backpropagation and parameter optimization
        dloss_dcoefs = tape.gradient(loss, coefs)
        #dloss_dcoefs_hidden = tape.gradient(loss, coefs_hidden)

        coefs.assign_sub(0.02*dloss_dcoefs)
        #coefs_hidden.assign_sub(0.02*dloss_dcoefs_hidden)
        loss_counter.append(tf.reduce_sum(loss))

    print(sum(loss_counter))
    loss_counter = []






tf.Tensor(248.9395619489717, shape=(), dtype=float64)
tf.Tensor(52.56347524304228, shape=(), dtype=float64)
tf.Tensor(27.078251684437042, shape=(), dtype=float64)
tf.Tensor(21.836935917935154, shape=(), dtype=float64)
tf.Tensor(19.96631876355519, shape=(), dtype=float64)
tf.Tensor(19.106939123537988, shape=(), dtype=float64)
tf.Tensor(18.648441301524972, shape=(), dtype=float64)
tf.Tensor(18.37610570715446, shape=(), dtype=float64)
tf.Tensor(18.199523099015483, shape=(), dtype=float64)
tf.Tensor(18.07589324071918, shape=(), dtype=float64)


TensorShape([5, 1, 81])